In [ ]:
####### Run a simple analysis of terrain around a selected location

# Step 1: Sign up for a Google Earth Engine Account here:
# https://earthengine.google.com/signup/

# you will use this to authenticate your account with GEE to run the script and
# utilize GEE layers

In [ ]:
# get any missing modules
! pip install earthengine-api geemap

In [1]:
# setup modules
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
import sys,os,os.path,tempfile

# spatial modules
import ee
#import geemap # currently not using as colorbars dont load properly in google collab
import geemap.colormaps as cm
import geemap.foliumap as geemap # use folium instead so coloramps work on goolge collab

# disable after updating
#geemap.update_package()

In [2]:
## setup output. defined dir or system temp dir
data_dir=os.path.expanduser(os.path.join(tempfile.gettempdir(),'terrain_analysis_output'))

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

print(" ")
print("****** Raster and blockage results output folder ******")
print("****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******")
print("****** Use the dir navigation to go up one level, then navigate to tmp/geo ******")
print(data_dir)

 
****** Raster and blockage results output folder ******
****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******
****** Use the dir navigation to go up one level, then navigate to tmp/geo ******
/var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output


In [3]:
##### initialize ee API  - you need to authenticate with GEE and initialize

## Trigger the authentication flow. You only need to do this once while running notebook
#ee.Authenticate()

## After inserting the API key initialize GEE
ee.Initialize()

In [5]:
### Define some GIS layers

# SEUS bounding box
bbox = ee.Geometry.Rectangle(-94.614744,29.317645,-74.441221,36.520919)

nexrad = ee.FeatureCollection('users/serbinsh/vector_data/amf3/nexrad_locations').filterBounds(bbox)
AandM_EC_Tower = ee.FeatureCollection('users/serbinsh/vector_data/amf3/BNF_AandM_EC_Tower')
potential_BNF_AOS_sites = ee.FeatureCollection('users/serbinsh/vector_data/amf3/potential_BNF_AOS_sites')
AandM_EC_Tower_2kmbuff = ee.FeatureCollection('users/serbinsh/vector_data/amf3/BNF_AandM_EC_Tower_2kmbuff_AlbersEA')
nexrad = ee.FeatureCollection('users/serbinsh/vector_data/amf3/nexrad_locations').filterBounds(bbox)
nexrad_100km = ee.FeatureCollection('users/serbinsh/vector_data/amf3/nexrad_100km_buffer_AEA').filterBounds(bbox)

In [10]:
# DEM - 1.3 Arc Second NED dataset
elev_dataset = ee.Image('USGS/NED').clip(bbox);
ned_elevation = elev_dataset.select('elevation')
dem_palette = cm.get_palette('Spectral_r', n_class=30)
demViz = {'min': 0.0, 'max': 2000.0, 'palette': dem_palette, 'opacity': 1}

# load NED 270m SRTM mTPI
mtpi_dataset = ee.Image('CSP/ERGo/1_0/US/mTPI').clip(bbox);
usMtpi = mtpi_dataset.select('elevation');
usMtpiVis = {'min': -200.0, 'max': 200.0, 'palette': ['0b1eff', '4be450', 'fffca4', 'ffa011', 'ff0000']};

# load 10m NED landforms
nedlandform_dataset = ee.Image('CSP/ERGo/1_0/US/landforms').clip(bbox);
landforms = nedlandform_dataset.select('constant');
landformsVis = {'min': 11.0, 'max': 42.0,'palette': [
    '141414', '383838', '808080', 'EBEB8F', 'F7D311', 'AA0000', 'D89382',
    'DDC9C9', 'DCCDCE', '1C6330', '68AA63', 'B5C98E', 'E1F0E5', 'a975ba',
    '6f198c']};

In [49]:
### create hillshade & slope
ned_hillshade = ee.Terrain.hillshade(ned_elevation)
ned_slope = ee.Terrain.slope(ned_elevation)
#ned_slope.getInfo()

In [52]:
### Show NED DEM data
srtm = geemap.Map(center=[33.315809,-85.198609], zoom=7)
srtm.addLayer(ned_elevation, demViz, 'Elevation above sea level');

srtm.addLayer(AandM_EC_Tower, {'color': 'black', 'strokeWidth': 1}, 'A & M Tower')
srtm.addLayer(AandM_EC_Tower_2kmbuff, {'color': 'black', 'opacity': 0.4}, 'A & M EC tower 2km buffer')
srtm.addLayer(nexrad,{'color': 'yellow', 'strokeWidth': 3},'NEXRAD locations')
srtm.addLayer(nexrad_100km, {'color': '#d3d3d3', 'opacity': 0.20}, 'NEXRAD 100km buffers')

srtm.add_colorbar(colors=demViz['palette'], vmin=demViz['min'], 
                  vmax=demViz['max'], label="Elevation (m)", 
                  caption="Elevation (m)",
                  layer_name="SRTM DEM", position="bottomright")


states = ee.FeatureCollection('TIGER/2018/States')
statesImage = ee.Image().paint(states, 0, 2)
srtm.addLayer(statesImage, {'palette': 'black'}, 'US States')
srtm.addLayerControl()
srtm

In [55]:
### Show hillshade
hillshade = geemap.Map(center=[33.315809,-85.198609], zoom=7)
hillshade.addLayer(ned_hillshade, {min: 150, max:255}, 'Hillshade');

hillshade.addLayer(AandM_EC_Tower, {'color': 'black', 'strokeWidth': 1}, 'A & M Tower')
hillshade.addLayer(AandM_EC_Tower_2kmbuff, {'color': 'black', 'opacity': 0.4}, 'A & M EC tower 2km buffer')
hillshade.addLayer(nexrad,{'color': 'yellow', 'strokeWidth': 3},'NEXRAD locations')
hillshade.addLayer(nexrad_100km, {'color': '#d3d3d3', 'opacity': 0.20}, 'NEXRAD 100km buffers')

states = ee.FeatureCollection('TIGER/2018/States')
statesImage = ee.Image().paint(states, 0, 2)
hillshade.addLayer(statesImage, {'palette': 'black'}, 'US States')
hillshade.addLayerControl()
hillshade

In [56]:
### Show slope
slope = geemap.Map(center=[33.315809,-85.198609], zoom=7)
slope_palette = cm.get_palette('Spectral_r', n_class=30)
slopeViz = {'min': 0.0, 'max': 35, 'palette': slope_palette, 'opacity': 1}
slope.addLayer(ned_slope, slopeViz, 'Slope (degrees)');

slope.add_colorbar(colors=slopeViz['palette'], vmin=slopeViz['min'], 
                  vmax=slopeViz['max'], label="Slope (degrees)", 
                  caption="Slope (degrees)",
                  layer_name="SRTM Slope", position="bottomright")

slope.addLayer(AandM_EC_Tower, {'color': 'black', 'strokeWidth': 1}, 'A & M Tower')
slope.addLayer(AandM_EC_Tower_2kmbuff, {'color': 'black', 'opacity': 0.4}, 'A & M EC tower 2km buffer')
slope.addLayer(nexrad,{'color': 'yellow', 'strokeWidth': 3},'NEXRAD locations')
slope.addLayer(nexrad_100km, {'color': '#d3d3d3', 'opacity': 0.20}, 'NEXRAD 100km buffers')


slope

In [57]:
### Some stats
#reducers = ee.Reducer.mean().combine({'reducer2': ee.Reducer.stdDev(),'sharedInputs': 'true'});

#geemap.zonal_statistics(dem, states, out_dem_stats, statistics_type='MEAN', scale=1000)



# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
out_dem_stats = os.path.join(data_dir, 'nexrad_slope_stats.csv')
#geemap.zonal_statistics(ned_slope, nexrad_100km, out_dem_stats, statistics_type='MEAN', scale=1000)
geemap.zonal_statistics(ned_slope, nexrad_100km, out_dem_stats, statistics_type='MEAN')

out_dem_stats = os.path.join(data_dir, 'AandM_slope_mean.csv')
geemap.zonal_statistics(ned_slope, AandM_EC_Tower_2kmbuff, out_dem_stats, statistics_type='MEAN')
out_dem_stats = os.path.join(data_dir, 'AandM_slope_minmax.csv')
geemap.zonal_statistics(ned_slope, AandM_EC_Tower_2kmbuff, out_dem_stats, statistics_type='MIN_MAX')

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output/nexrad_slope_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output/AandM_slope_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output/AandM_slope_minmax.csv
